In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

from plotly.subplots import make_subplots
import plotly.graph_objects as go

### Daily Model
1. Add when there's a signal
2. Potential crash/bear market when there's 2 signals for the past month\

In [2]:
# visualization, not quantifiable
symbol = "^TWII"
data = yf.download(symbol, start='2018-01-01', rounding=2, progress=False)
returns = data[["Close"]].pct_change()


returns["pct_power_5"] = np.power(returns.Close, 5)
tajm = 242
alpha = 2 / (tajm + 1)
returns['cumsum_bm'] = returns.Close.cumsum()
returns['pct_for_calculating_profit'] = returns.Close.shift(-2)

returns["pct_power_5_ema"] = returns["pct_power_5"].ewm(alpha=alpha, adjust=False).mean()
returns['pct_power_5_ema_shift_1'] = returns.pct_power_5_ema.shift(1)
returns.dropna(inplace=True)


returns["signal"] = np.where(
    (returns["pct_power_5_ema"] > returns["pct_power_5_ema_shift_1"] * 1.01) | 
    (returns["pct_power_5_ema"] > 0), 
    1, 
    -1
)


returns['diff_over_the_past_month'] = returns["Close"].rolling(5).sum()

k = 0
u = -1

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1,
                     subplot_titles=symbol)

fig.add_trace(go.Scatter(
    x=returns.index[k:u],
    y=returns.iloc[k:u].cumsum_bm,
    mode='lines',
    name='Close Price'
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=returns.index[k:u],
    y=returns.iloc[k:u].diff_over_the_past_month,
    mode='lines',
    name='RETURN OVER THE MONTH Price'
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=returns.index[k:u],
    y=returns.iloc[k:u]['signal'],
    mode='lines',
    name='Signal'
), row=2, col=1)

fig.update_layout(
    title=symbol,
    template="plotly_white",
    height=600
)

fig.update_xaxes(title_text="Index", row=2, col=1)
fig.update_yaxes(title_text="Signal", row=2, col=1)

fig.show()

In [3]:
symbol = "^TWII"
data = yf.download(symbol, start='2017-01-01', rounding=2, progress=False)

In [4]:
# params
tajm = 242
alpha = 2 / (tajm + 1)
dip_range = -0.08
dip_ready_for_buying_the_dip = False
trading_days_list = list(data.index)
adding_extreme = 0
extreme_last_for_days = 25
extreme_counter = extreme_last_for_days+1

adding_signal = 0
signal_threshold = 2
current_number_of_signal = 0
signal_last_for_days = 20
signal_counter = signal_last_for_days+1
signal_dict = {}
signal = 1
short_signal = 0
for index, da in enumerate(trading_days_list):
    # preprocess
    if index < tajm: continue
    if da.strftime("%Y-%m-%d") == '2020-09-24': 
        
        signal_dict[da] = 1; continue;
    df_day = data[:da].iloc[:, :].copy()



    df_day['pct'] = df_day['Close'].pct_change()
    # df_day['pct'] = (df_day['High']/df_day['Low'])/df_day['Low']
    df_day['power'] = np.power(df_day.pct, 5)
    df_day["power_ema"] = df_day["power"].ewm(alpha=alpha, adjust=False).mean()
    df_day['power_ema_shift1'] = df_day.power_ema.shift(1)
    
    df_day['diff_5d'] = df_day.pct.rolling(5).sum()
    df_day['diff_5d_diff'] = df_day['diff_5d'].diff()
    df_day['signal'] = np.where(
        (df_day["power_ema"] > df_day["power_ema_shift1"] * 1.01) | 
        (df_day["power_ema"] > 0), 
        1, 
        -1
    )
    df_day_last = df_day.tail(1)
    # dominating rule: {extreme_counter} > {extreme_last_for_days}
    if extreme_counter < extreme_last_for_days:
        extreme_counter += 1; 
        signal_dict[da] = signal
        print(f"*****Date: {da}, Signal: {signal}, Current Signals: {current_number_of_signal}, Signal Counter: {signal_counter}")
        
        continue;
    elif extreme_counter == extreme_last_for_days: 
        extreme_counter+=1; signal = 1;
    
    # extreme condition: 
    # buy the dip and last for {extreme_last_for_days} days
    if not dip_ready_for_buying_the_dip and df_day_last.diff_5d.values < dip_range:
        dip_ready_for_buying_the_dip = True;
        print("ready for plunge on {}".format(da));
    if dip_ready_for_buying_the_dip and df_day.diff_5d_diff.values[-1] > 0:
        signal = 1 + adding_extreme;
        extreme_counter = 0;
        print("**start buying the plunge on {}".format(da)); 
        print("============================")
        dip_ready_for_buying_the_dip = False
        signal_dict[da] = signal
        
        continue;
    
    if signal_counter < signal_last_for_days: 
        signal_counter += 1;
        signal_dict[da] = short_signal
        if df_day_last.signal.values == -1: signal_counter = 0;
        print(f"Date: {da}, last: {df_day_last.signal.values}, Signal: {signal}, Current Signals: {current_number_of_signal}, Signal Counter: {signal_counter}")

        continue;
    elif signal_counter == signal_last_for_days: 
        signal_counter += 1;
        signal = 1;
        current_number_of_signal = 0
        
    # Signal: 
    # nothing changed but 
    #   signal = 1 + {adding_first_singal} 
    # if the condition is satisfied
    if df_day_last.signal.values == -1:
        signal = 1 + adding_signal; 
        current_number_of_signal += 1;
    
    # two signals in {signal_last_for_days} days:
    # reset {signal_counter} to 1 instead of 2;
    if current_number_of_signal >= signal_threshold: 
        # print("start shorting at {}".format(da))
        current_number_of_signal = 1
        signal = short_signal;
        signal_counter = 0;
    print(f"Date: {da}, Signal: {signal}, Current Signals: {current_number_of_signal}, Signal Counter: {signal_counter}")

    signal_dict[da] = signal
    

Date: 2017-12-29 00:00:00, Signal: 1, Current Signals: 0, Signal Counter: 21
Date: 2018-01-02 00:00:00, Signal: 1, Current Signals: 0, Signal Counter: 21
Date: 2018-01-03 00:00:00, Signal: 1, Current Signals: 0, Signal Counter: 21
Date: 2018-01-04 00:00:00, Signal: 1, Current Signals: 0, Signal Counter: 21
Date: 2018-01-05 00:00:00, Signal: 1, Current Signals: 0, Signal Counter: 21
Date: 2018-01-08 00:00:00, Signal: 1, Current Signals: 0, Signal Counter: 21
Date: 2018-01-09 00:00:00, Signal: 1, Current Signals: 0, Signal Counter: 21
Date: 2018-01-10 00:00:00, Signal: 1, Current Signals: 1, Signal Counter: 21
Date: 2018-01-11 00:00:00, Signal: 1, Current Signals: 1, Signal Counter: 21
Date: 2018-01-12 00:00:00, Signal: 1, Current Signals: 1, Signal Counter: 21
Date: 2018-01-15 00:00:00, Signal: 1, Current Signals: 1, Signal Counter: 21
Date: 2018-01-16 00:00:00, Signal: 1, Current Signals: 1, Signal Counter: 21
Date: 2018-01-17 00:00:00, Signal: 1, Current Signals: 1, Signal Counter: 21

In [5]:
symbol = "0050.TW"
data = yf.download(symbol, start='2017-01-01', rounding=2, progress=False)
data['origin'] = df_day['signal']
data['signal'] = signal_dict
data['pct_for_calc'] = np.log(data['Open']/data['Open'].shift(1)).shift(2)
data['strategy_return'] = (data['pct_for_calc'] * data['signal']).cumsum()
data.dropna(inplace=True)
data['bm'] = data['pct_for_calc'].cumsum()
data['diff'] = data['strategy_return'] - data['bm']
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.1,
                     subplot_titles=symbol)


fig.add_trace(go.Scatter(
    x=data.index,
    y=data['strategy_return'],
    mode='lines',
    name='strat'
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=data.index,
    y=data['bm'],
    mode='lines',
    name='bm'
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=data.index,
    y=data['diff'],
    mode='lines',
    name='exceed return'
), row=1, col=1)


fig.add_trace(go.Scatter(
    x=data.index,
    y=data['signal'],
    mode='lines',
    name='Signal'
), row=2, col=1)

fig.add_trace(go.Scatter(
    x=data.index,
    y=data['origin'],
    mode='lines',
    name='origin'
), row=3, col=1)

fig.update_layout(
    title=symbol,
    template="plotly_white",
    height=800,
    width=1200
)


In [6]:
len(data['signal'].iloc[242:]), len(data['signal'].iloc[242:].dropna())

(1481, 1481)

### Monthly Model
1. Add to the position when there's a signal with abnormal plunges/dips
2. bear market when there's 2 signals for the past 3 months, reverse when there's abnormal dips

In [7]:
# visualization, not quantifiable - Monthly Version
symbol = "^TWII"
data = yf.download(symbol, start='2016-01-01', rounding=2, progress=False)
returns = data[["Close"]].pct_change().dropna()
returns = returns.resample("ME").sum()

returns["pct_power_5"] = np.power(returns.Close, 5)
tajm = 3
alpha = 2 / (tajm + 1)
returns['cumsum_bm'] = returns.Close.cumsum()
returns['ma_cumsum_bm'] = returns.cumsum_bm.rolling(60).mean()
returns['pct_for_calculating_profit'] = returns.Close.shift(-2)

returns["pct_power_5_ema"] = returns["pct_power_5"].ewm(alpha=alpha, adjust=False).mean()
returns['pct_power_5_ema_shift_1'] = returns.pct_power_5_ema.shift(1)

returns['signal'] = returns.apply(lambda row: 1 if row['pct_power_5_ema'] > row['pct_power_5_ema_shift_1']*1.01 or row['pct_power_5_ema'] > 0 else -1, axis=1)

returns['strat_pct'] = returns.signal * returns.pct_for_calculating_profit

k = 0
u = -1

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1,
                     subplot_titles=symbol)

fig.add_trace(go.Scatter(
    x=returns.index[k:u],
    y=returns.iloc[k:u].cumsum_bm,
    mode='lines',
    name='Close Price'
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=returns.index[k:u],
    y=returns.iloc[k:u]['signal'],
    mode='lines',
    name='Signal'
), row=2, col=1)

fig.update_layout(
    title=symbol,
    template="plotly_white",
    height=600
)

fig.update_xaxes(title_text="Index", row=2, col=1)
fig.update_yaxes(title_text="Signal", row=2, col=1)

fig.show()

### TWII to TWOII

In [8]:
import yfinance as yf
start = '2019-01-01'
twii = yf.download("^TWII", start=start, progress=False, rounding=2)
twoii = yf.download("^TWOII", start=start, progress=False, rounding=2)
df = twii[['Close']].copy()
df.columns = ["twii"]
df['twoii'] = twoii['Close']
df_log = np.log(df / df.shift(1))
df_log = df_log.resample("W-FRI").sum()

In [9]:
import plotly.graph_objs as go
import pandas as pd
import yfinance as yf
start = '2019-01-01'
twii = yf.download("^TWII", start=start, progress=False, rounding=2)
twoii = yf.download("^TWOII", start=start, progress=False, rounding=2)
df = twii[['Close']].copy()
df.columns = ["twii"]
df['twoii'] = twoii['Close']
df_log = np.log(df / df.shift(1))
df_log = df_log.resample("W-FRI").sum()
# Assuming df_log is your DataFrame with the columns 'twii' and 'twoii'
df_log['diff'] = df_log['twii'] - df_log['twoii']
df_cumsum = df_log.cumsum()


import plotly.graph_objs as go

# Create the Plotly figure
fig = go.Figure()

# Add the TWII trace (primary y-axis)
fig.add_trace(go.Scatter(x=df_cumsum.index, y=df_cumsum['twii'], mode='lines', name='TWII'))

# Add the TWOII trace (primary y-axis)
fig.add_trace(go.Scatter(x=df_cumsum.index, y=df_cumsum['twoii'], mode='lines', name='TWOII'))

# Add the Cumulative Sum of the Difference trace (secondary y-axis)
fig.add_trace(go.Scatter(x=df_cumsum.index, y=df_cumsum['diff'], mode='lines', name='Cumulative Sum of Difference', yaxis="y2"))

# Customize the layout
fig.update_layout(
    title="Cumulative Sum of Difference Between TWII and TWOII",
    xaxis_title="Date",
    yaxis_title="TWII / TWOII",
    yaxis2=dict(
        title="Cumulative Sum of Difference",
        overlaying="y",  # Overlay on the primary y-axis
        side="right"     # Place the secondary y-axis on the right
    ),
    template="plotly_dark"
)

# Show the plot
fig.show()


In [10]:
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import yfinance as yf
import numpy as np

symbol = "^TWII"
start = '2017-01-01'
data = yf.download(symbol, start=start, rounding=2, progress=False)

# Data for plot 1
data_plot1 = data[["Close"]].pct_change().dropna()
data_plot1["pct_power_5"] = np.power(data_plot1.Close, 5)
tajm = 242
alpha = 2 / (tajm + 1)
data_plot1['cumsum_bm'] = data_plot1.Close.cumsum()
data_plot1['ma_cumsum_bm'] = data_plot1.cumsum_bm.rolling(60).mean()
data_plot1['pct_for_calculating_profit'] = data_plot1.Close.shift(-2)
data_plot1["pct_power_5_ema"] = data_plot1["pct_power_5"].ewm(alpha=alpha, adjust=False).mean()
data_plot1['pct_power_5_ema_shift_1'] = data_plot1.pct_power_5_ema.shift(1)

data_plot1["signal"] = np.where(
    (data_plot1["pct_power_5_ema"] > data_plot1["pct_power_5_ema_shift_1"] * 1.01) | 
    (data_plot1["pct_power_5_ema"] > 0), 
    1, 
    -1
)
data_plot1['strat_pct'] = data_plot1.signal * data_plot1.pct_for_calculating_profit

# Data for plot 2 (monthly)
data_plot2 = data[["Close"]].pct_change().dropna()
data_plot2 = data_plot2.resample("ME").sum()

data_plot2["pct_power_5"] = np.power(data_plot2.Close, 5)
tajm = 3
alpha = 2 / (tajm + 1)
data_plot2['cumsum_bm'] = data_plot2.Close.cumsum()
data_plot2['ma_cumsum_bm'] = data_plot2.cumsum_bm.rolling(60).mean()
data_plot2['pct_for_calculating_profit'] = data_plot2.Close.shift(-2)
data_plot2["pct_power_5_ema"] = data_plot2["pct_power_5"].ewm(alpha=alpha, adjust=False).mean()
data_plot2['pct_power_5_ema_shift_1'] = data_plot2.pct_power_5_ema.shift(1)

data_plot2["signal"] = np.where(
    (data_plot2["pct_power_5_ema"] > data_plot2["pct_power_5_ema_shift_1"] * 1.01) | 
    (data_plot2["pct_power_5_ema"] > 0), 
    1, 
    -1
)

data_plot2['strat_pct'] = data_plot2.signal * data_plot2.pct_for_calculating_profit

# Data for plot 3 (TWII vs TWOII)
twoii = yf.download("^TWOII", start=start, progress=False, rounding=2)
data_plot3 = data[['Close']].copy()
data_plot3.columns = ["twii"]
data_plot3['twoii'] = twoii['Close']
df_log = np.log(data_plot3 / data_plot3.shift(1))
df_log = df_log.resample("W-FRI").sum()

df_log['diff'] = df_log['twii'] - df_log['twoii']
df_cumsum = df_log.cumsum()

# Create subplots with reduced vertical spacing
fig = make_subplots(rows=5, cols=1, shared_xaxes=True, vertical_spacing=0.05,
                     subplot_titles=[symbol, symbol, symbol, symbol, symbol])

# Add plot 1
fig.add_trace(go.Scatter(x=data_plot1.index, y=data_plot1.cumsum_bm, mode='lines', name='Close Price'), row=1, col=1)
fig.add_trace(go.Scatter(x=data_plot1.index, y=data_plot1['signal'], mode='lines', name='Signal for plot1'), row=2, col=1)

# Add plot 2
fig.add_trace(go.Scatter(x=data_plot2.index, y=data_plot2.cumsum_bm, mode='lines', name='Close Price'), row=3, col=1)
fig.add_trace(go.Scatter(x=data_plot2.index, y=data_plot2['signal'], mode='lines', name='Signal for plot2'), row=4, col=1)

# Add plot 3
fig.add_trace(go.Scatter(x=df_cumsum.index, y=df_cumsum['twii'], mode='lines', name='TWII'), row=5, col=1)
fig.add_trace(go.Scatter(x=df_cumsum.index, y=df_cumsum['twoii'], mode='lines', name='TWOII'), row=5, col=1)
fig.add_trace(go.Scatter(x=df_cumsum.index, y=df_cumsum['diff'], mode='lines', name='DIFF', yaxis="y2"), row=5, col=1)

# Update layout
fig.update_layout(
    title=symbol,
    template="plotly_white",
    height=1800
)

# Show the plot and save it as an HTML file
fig.write_html("combined_plot.html")
fig.show()


### insert into DB

In [11]:
# import psycopg2
# pg_host = '10.188.200.16'
# pg_port = '5432'
# pg_user = "jimmy"
# pg_passwd = "123123"
# conn = psycopg2.connect(database = "tw", user = pg_user, password = pg_passwd, host= pg_host, port = pg_port)
# cur = conn.cursor()


# ins_sql99 = ""
# df_insert = pd.DataFrame(list(signal_dict.items()), columns=['Date', 'Value'])
# df_insert['Value'] = df_insert['Value'].shift(1)
# df_insert = df_insert.iloc[:244]
# df_insert.dropna(inplace=True)
# for index,  row in df_insert.iterrows():
#     ins_sql99 += f" INSERT INTO model_strategy_stocks_and_weight(DA, STRATEGY_NAME, CODE, WEIGHT) " \
#                  f" VALUES('{row['Date']}', '0050_5D', '0050 TT Equity', '{row['Value']}') " \
#                  f" ON CONFLICT ON CONSTRAINT model_strategy_stocks_and_weight_pkey DO NOTHING;"
                 
# cur.execute(ins_sql99)
# conn.commit()


In [12]:
# sqlStr =    f" INSERT INTO strategy_entcontracts(strategy_name, cname, entno, market_name, display_name, strategy_description, trading_logic) " \
#             f" VALUES('0050_5D', '台股0050低波動策略', '00007', 'tw_market_etf', '台股ETF量化策略', '針對0050的多空訊號', '利用台灣加權指數日收益率，計算波動度並以其為訊號') " \
#             f" ON CONFLICT ON CONSTRAINT strategy_entcontracts_pkey DO NOTHING;"
# cur.execute(sqlStr)
# conn.commit()
